## Daniel Alonso, UID: 100444499

<img src='./images/1.png'>

<img src='./images/2.png'>

## Problem 1-a

Let $i$ be the source of a movement and $j$ be the destination

Let $x$ be movements from one node to another, where $x_{i,j}$ represents the movement from node $i$ to node $j$

Integer optimization formulation:

minimize $44x_{1,2} + 18x_{1,3} + 85x_{1,4} + 17x_{2,5} + 13x_{2,6} + 55x_{3,5} + 53x_{3,6} + 7x_{3,7} + 26x_{4,6} + 66x_{4,7} + 6x_{5,8} + 7x_{5,9} + 26x_{6,8} + 66x_{6,9} + 26x_{6,10} + 6x_{7,9} + 60x_{7,10} + 58x_{8,11} + 70x_{9,11} + 27x_{10,11}$

subject to:

$x_{1,2} + x_{1,3} + x_{1,4} = 1$

$x_{1,2} - x_{1,5} - x_{1,6} = 0 $

$x_{1,3} - x_{3,5} - x_{3,6} - x_{3,7} = 0$

$x_{1,4} - x_{4,6} - x_{4,7} = 0$

$x_{2,5} + x_{3,5} - x_{5,8} - x_{5,9} = 0$

$x_{2,6} + x_{3,6} + x_{4,6} - x_{6,8} - x_{6,9} - x_{6,10} = 0$

$x_{3,7} + x_{4,7} - x_{7,9} - x_{7,10} = 0$

$x_{5,8} + x_{6,8} - x_{8,11} = 0$

$x_{5,9} + x_{6,9} + x_{7,9} - x_{9,11} = 0$

$x_{6,10} + x_{7,10} - x_{10,11} = 0$

$x_{8,11} + x_{9,11} + x_{10,11} = 1$

The integer optimization formulation (in my opinion) is more appropriate. It not only will provide the optimal integer solution but it also describes the problem better and more accurately.

## Problem 1-b

In [11]:
# Gurobi-python implementation
from gurobipy import *

In [14]:
years = range(0,20)
projects = range(0,11)

In [15]:
# primal objective coefficients
r_coeff = [44, 18, 85, 17, 13, 55, 53, 7, 26, 66, 6, 7, 26, 66, 26, 6, 60, 58, 70, 27] 

# left-hand side (LHS) coefficients (matrix A)
A_coeff = [[1, 1, 1],
           [1, -1, -1],
           [1, -1, -1, -1],
           [1, -1, -1],
           [1, 1, -1, -1],
           [1, 1, 1, -1, -1, -1],
           [1, 1, -1, -1],
           [1, 1, -1],
           [1, 1, 1, -1],
           [1, 1, -1],
           [1, 1, 1]]

# right-hand side (RHS) coefficients
b_coeff = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

r = {j : r_coeff[j] for j in projects}

A = {i : {j : A_coeff[i][j] for j in projects} 
    for i in years}

b = {i : b_coeff[i] for i in years}

model = Model('projectsip1')

x = model.addVars(projects, name="x", vtype=GRB.BINARY)

# Capacity constraints
model.addConstrs((quicksum(A[i][j] * x[j] for j in projects)
                           <= b[i] 
                            for i in years))

# Variable upper bound constraints
model.addConstrs((x[j] <= 1 for j in projects))

# Objective
obj = quicksum(r[j] * x[j] for j in projects)

model.setObjective(obj, GRB.MINIMIZE)

# disable Presolve
model.setParam(GRB.Param.Presolve, 0)
# disable Heuristics
model.setParam(GRB.Param.Heuristics, 0)
# disable Cuts
model.setParam(GRB.Param.Cuts, 0)
        
model.optimize()

# Display solution (print the name of each variable and the solution value)
print('--------------------------------')
print('\nOptimal solution:\n')

print('Variable Information:')
                 
for v in model.getVars():
    print("%s %s %8.2f" % 
              (v.Varname, "=", v.X))
    
    print(" ")
        
print('\nOptimal objective value: %g' % model.objVal)


IndexError: list index out of range

## Problem 1-c

<img src='./images/3.png'>

## Problem 2-a

## Problem 2-b

In [16]:
years = range(0,3)
projects = range(0,4)

In [ ]:
# primal objective coefficients
r_coeff = [44, 18, 85, 17, 13, 55, 53, 7, 26, 66, 6, 7, 26, 66, 26, 6, 60, 58, 70, 27] 

# left-hand side (LHS) coefficients (matrix A)
A_coeff = [[1, 1, 1],
           [1, -1, -1],
           [1, -1, -1, -1],
           [1, -1, -1],
           [1, 1, -1, -1],
           [1, 1, 1, -1, -1, -1],
           [1, 1, -1, -1],
           [1, 1, -1],
           [1, 1, 1, -1],
           [1, 1, -1],
           [1, 1, 1]]

# right-hand side (RHS) coefficients
b_coeff = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

r = {j : r_coeff[j] for j in projects}

A = {i : {j : A_coeff[i][j] for j in projects} 
    for i in years}

b = {i : b_coeff[i] for i in years}

model = Model('projectsip1')

x = model.addVars(projects, name="x", vtype=GRB.BINARY)

# Capacity constraints
model.addConstrs((quicksum(A[i][j] * x[j] for j in projects)
                           <= b[i] 
                            for i in years))

# Variable upper bound constraints
model.addConstrs((x[j] <= 1 for j in projects))

# Objective
obj = quicksum(r[j] * x[j] for j in projects)

model.setObjective(obj, GRB.MINIMIZE)

# disable Presolve
model.setParam(GRB.Param.Presolve, 0)
# disable Heuristics
model.setParam(GRB.Param.Heuristics, 0)
# disable Cuts
model.setParam(GRB.Param.Cuts, 0)
        
model.optimize()

# Display solution (print the name of each variable and the solution value)
print('--------------------------------')
print('\nOptimal solution:\n')

print('Variable Information:')
                 
for v in model.getVars():
    print("%s %s %8.2f" % 
              (v.Varname, "=", v.X))
    
    print(" ")
        
print('\nOptimal objective value: %g' % model.objVal)
